In [20]:
#import module

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import SimpleRNN, Embedding, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model

1. 전처리(Preprocessing)

In [21]:
#Tokenizer() : 토큰화와 정수 인코딩을 위해 사용

tokenizer = Tokenizer()
train_text = "The earth is an awesome place live"

# 단어 집합 생성
tokenizer.fit_on_texts([train_text])

# 정수 인코딩
sub_text = "The earth is an great place live"
sequences = tokenizer.texts_to_sequences([sub_text])[0]

print("정수 인코딩 : ",sequences)
print("단어 집합 : ",tokenizer.word_index)


정수 인코딩 :  [1, 2, 3, 4, 6, 7]
단어 집합 :  {'the': 1, 'earth': 2, 'is': 3, 'an': 4, 'awesome': 5, 'place': 6, 'live': 7}


In [22]:
# pad_sequence() : 서로 다른 길이의 샘플들의 길이를 동일하게 맞춰주는 작업
# 첫번쨰 인자 : 패딩을 진행할 데이터 / maxlen : 정규화 할 길이 / padding : 'pre'(앞 에 0을 채움) or 'post' (뒤에 0을 채움)
pad_sequences([[1, 2, 3], [3, 4, 5, 6], [7, 8]], maxlen=3, padding='pre')

array([[1, 2, 3],
       [4, 5, 6],
       [0, 7, 8]])

2. 워드 임베딩(Word Embedding)

In [24]:
# Embedding() : 단어를 밀집 벡터(Dense Vector)로 만드는 역할을 한다.
# 첫번째 인자 : 단어 집합의 크기. 즉, 총 단어의 개수 / 두번째 인자 : 임베딩 벡터의 출력 차원. 결과로서 나오는 임베딩 벡터의 크기 : input_length = 입력 시퀀스의 길이

# 1. 토큰화
tokenized_text = [['Hope', 'to', 'see', 'you', 'soon'], ['Nice', 'to', 'see', 'you', 'again']]

# 2. 각 단어에 대한 정수 인코딩
encoded_text = [[0, 1, 2, 3, 4],[5, 1, 2, 3, 6]]

# 3. 위 정수 인코딩 데이터가 아래의 임베딩 층의 입력이 된다.
vocab_size = 7
embedding_dim = 2
Embedding(vocab_size, embedding_dim, input_length=5)

""" 각 정수는 아래의 테이블의 인덱스로 사용되며 Embedding()은 각 단어마다 임베딩 벡터를 리턴한다.
+------------+------------+
|   index    | embedding  |
+------------+------------+
|     0      | [1.2, 3.1] |
|     1      | [0.1, 4.2] |
|     2      | [1.0, 3.1] |
|     3      | [0.3, 2.1] |
|     4      | [2.2, 1.4] |
|     5      | [0.7, 1.7] |
|     6      | [4.1, 2.0] |
+------------+------------+

# 위의 표는 임베딩 벡터가 된 결과를 예로서 정리한 것이고 Embedding()의 출력인 3D 텐서를 보여주는 것이 아님. """


' 각 정수는 아래의 테이블의 인덱스로 사용되며 Embedding()은 각 단어마다 임베딩 벡터를 리턴한다.\n+------------+------------+\n|   index    | embedding  |\n+------------+------------+\n|     0      | [1.2, 3.1] |\n|     1      | [0.1, 4.2] |\n|     2      | [1.0, 3.1] |\n|     3      | [0.3, 2.1] |\n|     4      | [2.2, 1.4] |\n|     5      | [0.7, 1.7] |\n|     6      | [4.1, 2.0] |\n+------------+------------+\n\n# 위의 표는 임베딩 벡터가 된 결과를 예로서 정리한 것이고 Embedding()의 출력인 3D 텐서를 보여주는 것이 아님. '

3. 모델링(Modeling)

In [25]:
#Sequential() : 층을 구성하는 방법 중 하나로 층을 순서대로 쌓아서 구성하는 방법

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length = 5)) #임베딩 층을 추가하는 예시

In [26]:
model = Sequential()
model.add(Dense(1, input_dim = 3, activation='relu')) # 전결합층(fully-connected layer)을 추가하는 예시

In [27]:
model = Sequential()
model.add(Dense(8, input_dim = 4,activation='relu'))
model.add(Dense(1, activation='sigmoid')) # 출력층을 추가하는 예시(이전 층의 뉴런수가 1이므로 input_dim을 적지 않아도 됨)

In [28]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 8)                 40        
                                                                 
 dense_9 (Dense)             (None, 1)                 9         
                                                                 
Total params: 49 (196.00 Byte)
Trainable params: 49 (196.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


4. 컴파일(Compile)과 훈련(Training)

In [29]:
#compile() : 모델을 기계가 이해할 수 있도록 컴파일
#optimizer = 훈련 과정을 설정하는 옵티마이저를 설정
#loss = 훈련 과정에서 사용할 손실 함수를 설정
#metrics = 훈련을 모니터링하기 위한 지표를 설정

vocab_size = 10000
embedding_dim = 32
hidden_units = 32

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(SimpleRNN(hidden_units))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

In [30]:
# 위의 compile() 코드의 연장선상인 코드
# fit() : 모델을 학습
X_train = [[1, 2, 3, 4, 5], [6, 7, 8, 9, 10]]
y_train = [1, 0]
model.fit(X_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
1/1 [==============================] - 1s 719ms/step - loss: 0.6763 - acc: 1.0000
Epoch 2/10
1/1 [==============================] - 0s 7ms/step - loss: 0.6034 - acc: 1.0000
Epoch 3/10
1/1 [==============================] - 0s 0s/step - loss: 0.5506 - acc: 1.0000
Epoch 4/10
1/1 [==============================] - 0s 17ms/step - loss: 0.5025 - acc: 1.0000
Epoch 5/10
1/1 [==============================] - 0s 0s/step - loss: 0.4563 - acc: 1.0000
Epoch 6/10
1/1 [==============================] - 0s 2ms/step - loss: 0.4115 - acc: 1.0000
Epoch 7/10
1/1 [==============================] - 0s 0s/step - loss: 0.3684 - acc: 1.0000
Epoch 8/10
1/1 [==============================] - 0s 18ms/step - loss: 0.3276 - acc: 1.0000
Epoch 9/10
1/1 [==============================] - 0s 0s/step - loss: 0.2895 - acc: 1.0000
Epoch 10/10
1/1 [==============================] - 0s 19ms/step - loss: 0.2546 - acc: 1.0000


5. 평가(Evaluation)와 예측(Prediction)

In [31]:
# evaluate() : 테스트 데이터를 통해 학습한 모델에 대한 정확도를 평가
X_test = [[1, 2, 3, 4, 5], [6, 7, 8, 9, 10]]
y_test = [1, 0]
model.evaluate(X_test, y_test, batch_size=32)

1/1 [==============================] - 0s 131ms/step - loss: 0.2232 - acc: 1.0000


[0.22317306697368622, 1.0]

In [32]:
# predict() : 임의의 입력에 대한 모델의 출력값을 확인
model.predict(X_test, batch_size=32)

1/1 [==============================] - 0s 101ms/step


array([[0.8028177 ],
       [0.20285487]], dtype=float32)

6. 모델의 저장(Save)과 로드(Load)

In [33]:
#save() : 인공 신경망 모델을 hdf5 파일에 저장
# model.save("model_name.h5")

c:\Users\winte\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [35]:
#load_model() : 저장해둔 모델을 불러옴
# model = load_model("model_name.h5")

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xd5 in position 29: invalid continuation byte